<div style="display: flex; gap: 10px;">
  <img src="../images/HOOPS_AI.jpg" style="width: 20%;">
</div>

# HOOPS AI: CAD Access Module

## Overview

The `cadaccess` module provides a python interface for accessing and manipulating CAD (Computer-Aided Design) models this is empowered by HOOPS Exchange.
It defines a set of methods for loading, inspecting, and processing CAD files.

Set the license, with option validate == True. This internally test the license to confirm all is set up properly!

In [1]:
import hoops_ai
import os

hoops_ai.set_license(hoops_ai.use_test_license(), validate=True)
#hoops_ai.set_license("my-license-key")

ℹ️ Using TEST LICENSE (expires December 8, 2025 - 40 days remaining)
   For production use, obtain your own license from Tech Soft 3D
✓ HOOPS AI License: Valid (TEST LICENSE - expires Dec 6, 2025)


In [ ]:
import pathlib


nb_dir = pathlib.Path.cwd()
output_dir          = nb_dir.joinpath("out")


# Create output directory if it does not exist
if not output_dir.exists():
    os.makedirs(output_dir)

### HOOPSLoader 

The `HOOPSLoader` interface defines methods for loading CAD files. For now the generla options implies read feature and read solid, but this APi will get richer for each version.

Key methods:
- `create_from_file(self, filename: str) -> CADModel`: Loads a CAD file and returns a HOOPSModel
- `get_model(self) -> HOOPSModel`: Returns the loaded HOOPSModel
- `set_general_options(self, options: Dict[str, Any]) -> None`: Sets options for loading CAD files
- `get_general_options(self) -> Dict[str, Any]`: Gets the current loading options

In [ ]:
from hoops_ai.cadaccess import HOOPSLoader, HOOPSModel, HOOPSTools
#--------------------------------------------------------------------------------------
# CAD Loader
#--------------------------------------------------------------------------------------
loader = HOOPSLoader()
    
general_options = loader.get_general_options()
print("default parameters ->", general_options)

general_options["read_feature"] = True 
general_options["read_solid"]   = True 

loader.set_general_options(general_options)
general_options = loader.get_general_options()
print("defined parameters ->", general_options)

### HOOPSModel

The `HOOPSModel` interface represents a loaded CAD model, providing access to its internal representation throught methods to get access to the BRep or Mesh:

Key methods:
- `get_filesource(self) -> str`: Returns the source file path
- `get_body(self, body_index: int = 0) -> Any`: Returns the body object from the model
- `get_brep(self, body_index: int = 0) -> BrepAccess`: Returns a BrepAccess implementation for the model
- `get_mesh(self, body_index: int = 0) -> MeshAccess`: Returns a MeshAccess implementation for the model (not yet implemented)
- `get_body_count(self) -> int`: Returns the number of bodies in the model

In [ ]:

cad_file_with_holes = nb_dir.parent.joinpath("packages", "cadfiles","pn_verschr_r1.prt.1")
model = loader.create_from_file(str(cad_file_with_holes))

In [ ]:
print(model)

The `HOOPSLoader` sets the general options of how to open the CAD file. The `HOOPSModel` contains a cad file accoridng to those specifications and now, the `HOOPSTools` will define how the Brep of this model is built.

### HOOPSTools 

The `HOOPSTools` interface provides utility methods for working with loaded CAD models:

Key methods:
- `brep_options(self) -> Dict[str, Any]`: Gets options for B-rep processing
- `adapt_brep(self, model: CADModel, brep_options: Dict[str, Any]) -> None`: Adapts the B-rep model according to provided options

- `tesselation_options(self) -> Dict[str, Any]`: Gets options for tessellation (not yet implemented)
- `adapt_tesselation(self, model: CADModel, tess_options: Dict[str, Any]) -> None`: Adapts the tessellation according to provided options (not yet implemented)

- `exportOBJ(self, model: CADModel, filepath: str) -> None`: Exports the model to OBJ format
- `exportSTEP(self, model: CADModel, filepath: str, overwrite: bool = False) -> None`: Exports the model to STEP format
- `exportStreamCache(self, model: CADModel, filepath: str, is_white_background: bool = True, overwrite: bool = False) -> None`: Exports the model to StreamCache format
- `extract_holes(self, model: CADModel) -> Dict[str, Any]`: Extracts information about holes in the model (only available for files CATIA, SLW, NX, Creo)

In [ ]:
tools = HOOPSTools()

tools.exportStreamCache(model, str(output_dir.joinpath("exported_cad_ref1")), is_white_background=True, overwrite=True)   

# Display a PNG image from file
from IPython.display import Image, display
display(Image(filename=str(output_dir.joinpath("exported_cad_ref1_white.png"))))

# Holes extraction for certain Files.

The HOOPs tool have a method to extract the holes type and the Brep face list. Available only for certain files this method could be used to help at tagging for machine learning task including holes. 

IDEA: get labeled data from CAD files CATIA, SLW, NX, Creo and then use the learned model to infere in other cad files.

In [ ]:
holes_data = tools.extract_holes(model)
print(holes_data)

using the HOOPS AI Insights module, you  may reuse the stream cache file generated to visualize the cad model.

Alternatively, you could export the cad file to step or obj.

Now we will show how to visualize the brep faces belonging to the GENERAl hole feature:


![](../packages/cadfiles/hole_extracted_from_tools.png)

# Interactive 3D Visualization with HOOPS AI Insights

The `insights` module provides a simple, Pythonic interface for visualizing CAD models in Jupyter notebooks.

## Three Ways to Visualize

The insights module provides three approaches to display the visualization from simplest to most advanced:
1. **Quick View** - One-liner for instant visualization
2. **Pre-converted SCS** - Reuse previously exported StreamCache files
3. **Full Control** - Manual viewer management for advanced workflows

## Load Pre-converted SCS Files (Efficient)

If you've already exported to StreamCache format (like we did earlier with HOOPSTools), reuse it to avoid re-conversion.

In [ ]:
from hoops_ai.insights import CADViewer

# Load the SCS file we exported earlier (faster - no conversion needed)
viewer = CADViewer(static_folder=output_dir, display_mode='sidecar')
viewer.load_scs_file(str(output_dir.joinpath("exported_cad_ref1.scs")))


In [ ]:
viewer.show()
# if the side car windows is too small, first resize the window to the desired size and then relaunch viewer.show()

In [ ]:
# Check viewer status
status = viewer.get_status()
print(f"Viewer is running on port: {status['port']}")
print(f"Viewer URL: {status['viewer_url']}")

---

## Working with the Viewer - Interactive Features

Let's use the first viewer to demonstrate interactive features. All viewers support the same API.

### 1. Highlight Specific Faces by Index

Highlight the faces that belong to the first hole feature we extracted earlier.

In [ ]:
# Highlight faces belonging to the first hole feature (default color)
viewer.set_face_color(holes_data['hole_features'][0]['face_indexes'], [255, 0, 0])

### 2. Clear Face Colors

Remove all colors to return to the default appearance.

In [ ]:
# Clear all face colors
viewer.clear_face_colors()


### 3. Get User-Selected Faces

**Interactive Step:**
1. Click on a face in the 3D viewer to highlight it
2. Use `Ctrl + Click` to highlight multiple faces
3. Run the cell below to retrieve your selection

In [ ]:
# Get the face indices you selected interactively
selected_faces = viewer.get_selected_faces()

print(f"You selected {len(selected_faces)} faces: {selected_faces}")

### 4. Highlight Selected Faces with Custom Color

Color your selected faces with a custom RGB color.

In [ ]:
# Highlight selected faces with a custom green color
viewer.set_face_color(selected_faces, [150, 255, 100])

---

## Cleanup

When done with a viewer, terminate it to free up the port and resources.

**Note:** If you used the context manager pattern (Approach 3), cleanup is automatic - no need to call `terminate()`.

In [ ]:
# Terminate viewers that weren't used with context manager
viewer.terminate()


print("All viewers terminated. Ports are now free for reuse.")

---

## Summary: Which Approach to Use?

| Approach | Best For | Pros | Cons |
|----------|----------|------|------|
| **Quick View** | Fast exploration, demos | One line, simplest | Less control |
| **Pre-converted SCS** | Repeated visualization | Fast (no conversion) | Requires SCS file |
| **Context Manager** | Production code, complex workflows | Auto-cleanup, full control | More verbose |

### Quick Reference

```python
# One-liner
from hoops_ai.insights import quick_view
viewer = quick_view("model.step")

# Reuse SCS file
from hoops_ai.insights import CADViewer
viewer = CADViewer(static_folder=output_dir)
viewer.load_scs_file("model.scs")
viewer.show()

# Context manager (recommended)
with CADViewer() as viewer:
    viewer.load_cad_file("model.step")
    viewer.show()
    # Automatic cleanup
```

### Key Methods

- `load_cad_file(path)` - Load CAD file (auto-converts to SCS)
- `load_scs_file(path)` - Load pre-converted SCS file (faster)
- `show()` - Display the viewer
- `set_face_color(indices, [r, g, b])` - Highlight faces
- `get_selected_faces()` - Get user-selected faces
- `clear_face_colors()` - Remove all colors
- `terminate()` - Clean up (not needed with context manager)